In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from pandas import Series,DataFrame

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

In [ ]:
random_seed = 1111

In [ ]:
#读取onehot编码后的分类数据
train_cat_after_oh = pd.read_csv('G:/Redhat_share/ML_micro/train_cat_after_oh.csv')
test_cat_after_oh = pd.read_csv('G:/Redhat_share/ML_micro/test_cat_after_oh.csv')
#读取数值型数据
train_only_num = pd.read_csv('G:/Redhat_share/ML_micro/train_only_num.csv')
test_only_num = pd.read_csv('G:/Redhat_share/ML_micro/test_only_num.csv')
#合并分类数据和数值数据
train_x = pd.merge(train_cat_after_oh,train_only_num,on='uid')
test_x = pd.merge(test_cat_after_oh,test_only_num,on='uid')
#读取训练数据y
train_y = pd.read_csv("G:/Redhat_share/ML_micro/train_y.csv")
#合并训练集的x和y
train_xy = pd.merge(train_x,train_y,on='uid')
train_xy.to_csv('G:/Redhat_share/ML_micro/train_xy.csv', index=False)
test_uid = test_x.uid

In [ ]:
train_xy.head()

In [ ]:
train_xy = train_xy.drop(['uid'],axis=1)
train,val = train_test_split(train_xy, test_size = 0.3,random_state=random_seed)
train_y = train.y
train_x = train.drop(['y'],axis=1)
val_y = val.y
val_x = val.drop(['y'],axis=1)
test_x = test_x.drop('uid',axis=1)

In [ ]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x,label=val_y)
dtest = xgb.DMatrix(test_x)
params={
    'booster':'gbtree',
    'early_stopping_rounds':100,
    'scale_pos_weight': 5,
    'objective': 'binary:logistic',
    'colsample_bytree':0.4,
    
    'gamma':0.05,
    'lambda':600,
    'max_depth':8,
    'subsample':0.6,
    'min_child_weight':3,
    'eta': 0.02,
    'seed':random_seed,
    'nthread':16,
    'eval_metric': 'auc'
    }

In [ ]:
watchlist  = [(dval,'val'), (dtrain,'train')]
model = xgb.train(params,dtrain,num_boost_round=5000,evals=watchlist)
model.save_model('xgb.model')
#预测
test_y = model.predict(dtest,ntree_limit=model.best_ntree_limit)
test_result = pd.DataFrame(columns=["uid","score"])
test_result.uid = test_uid
test_result.score = test_y
test_result.to_csv("result.csv",index=None,encoding='utf-8')